# 投資信託の評価

In [22]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import yfinance as yf

In [84]:
path = '../../../../data/row/investment_trust.parquet'
df = pl.read_parquet(path)
# 月末のデータを抽出
df = df.with_columns(
    (pl.col('date').dt.month_end()).alias('is_month_end')
).drop(
    ['ifree_em_bond_net_assets', 'sbi_em_stock_net_assets', 'sbi_vti_net_assets', 'emaxis_slim_dev_bond_net_assets']
)
month_end_dates = df['is_month_end'].unique().sort()
df_month = df.filter(pl.col('date').is_in(month_end_dates))
df_month = df_month.with_columns(
    pl.col('date').dt.strftime('%Y-%m').alias('date')
).drop('is_month_end')
df_month.head()

date,ifree_em_bond_nav,sbi_em_stock_nav,sbi_vti_nav,emaxis_slim_dev_bond_nav
str,f32,f32,f32,f32
"""2016-09""",9888.0,null,null,null
"""2016-10""",10135.0,null,null,null
"""2016-11""",10141.0,null,null,null
"""2017-01""",10652.0,null,null,null
"""2017-02""",10809.0,null,null,9997.0


In [85]:
ticker = yf.Tickers(['VGK', 'VPL'])
historical_data = ticker.history(period='max', interval='1mo')
historical_data.head()
df = pl.DataFrame({
    'date': pl.Series(historical_data.index).dt.strftime('%Y-%m'),
    'vgk_value': pl.Series(historical_data['Close']['VGK']),
    'vpl_value': pl.Series(historical_data['Close']['VPL']),
})
# df = df.with_columns([
#     pl.col('vgk_value').log().alias('vgk_log_value'),
#     pl.col('vpl_value').log().alias('vpl_log_value'),
# ])
# df = df.with_columns([
#     pl.col('vgk_log_value').diff().alias('vgk_log_return'),
#     pl.col('vpl_log_value').diff().alias('vpl_log_return'),
# ])

df.head()

[*********************100%***********************]  2 of 2 completed


date,vgk_value,vpl_value
str,f64,f64
"""2005-04""",23.385565,27.8941
"""2005-05""",23.448658,27.702244
"""2005-06""",23.764088,28.2313
"""2005-07""",24.419207,28.708014
"""2005-08""",25.113153,30.423054


## 各リターン，ボラティリティ

In [72]:
fund_name = 'sbi_vti_nav'
fund = df_month[['date', f'{fund_name}']]
fund = fund.with_columns([
    pl.col(f'{fund_name}').log().alias('log_value'),
])
fund = fund.with_columns([
    pl.col('log_value').diff().alias('log_diff'),
])
fund.head()

years = len(fund) / 12
first_value = fund[f'{fund_name}'].drop_nulls()[0]
last_value = fund[f'{fund_name}'].drop_nulls()[-1]
cagr = (last_value / first_value) ** (1 / years) - 1
print(f'年率リターン: {cagr:.2%}')

vti_vol = np.std(fund['log_diff'].drop_nulls().to_numpy()) * np.sqrt(12)
print(f'ボラティリティ: {vti_vol:.2%}')

sharp_ratio = cagr / vti_vol
print(f'シャープレシオ: {sharp_ratio:.2f}')

年率リターン: 12.99%
ボラティリティ: 17.99%
シャープレシオ: 0.72


In [69]:
fund_name = 'sbi_em_stock_nav'
fund = df_month[['date', f'{fund_name}']]
fund = fund.with_columns([
    pl.col(f'{fund_name}').log().alias('log_value'),
])
fund = fund.with_columns([
    pl.col('log_value').diff().alias('log_diff'),
])
fund.head()

years = len(fund) / 12
first_value = fund[f'{fund_name}'].drop_nulls()[0]
last_value = fund[f'{fund_name}'].drop_nulls()[-1]
cagr = (last_value / first_value) ** (1 / years) - 1
print(f'年率リターン: {cagr:.2%}')

vti_vol = np.std(fund['log_diff'].drop_nulls().to_numpy()) * np.sqrt(12)
print(f'ボラティリティ: {vti_vol:.2%}')

sharp_ratio = cagr / vti_vol
print(f'シャープレシオ: {sharp_ratio:.2f}')

年率リターン: 8.05%
ボラティリティ: 20.04%
シャープレシオ: 0.40


In [70]:
fund_name = 'emaxis_slim_dev_bond_nav'
fund = df_month[['date', f'{fund_name}']]
fund = fund.with_columns([
    pl.col(f'{fund_name}').log().alias('log_value'),
])
fund = fund.with_columns([
    pl.col('log_value').diff().alias('log_diff'),
])
fund.head()

years = len(fund) / 12
first_value = fund[f'{fund_name}'].drop_nulls()[0]
last_value = fund[f'{fund_name}'].drop_nulls()[-1]
cagr = (last_value / first_value) ** (1 / years) - 1
print(f'年率リターン: {cagr:.2%}')

vti_vol = np.std(fund['log_diff'].drop_nulls().to_numpy()) * np.sqrt(12)
print(f'ボラティリティ: {vti_vol:.2%}')

sharp_ratio = cagr / vti_vol
print(f'シャープレシオ: {sharp_ratio:.2f}')

年率リターン: 6.08%
ボラティリティ: 6.13%
シャープレシオ: 0.99


In [71]:
fund_name = 'ifree_em_bond_nav'
fund = df_month[['date', f'{fund_name}']]
fund = fund.with_columns([
    pl.col(f'{fund_name}').log().alias('log_value'),
])
fund = fund.with_columns([
    pl.col('log_value').diff().alias('log_diff'),
])
fund.head()

years = len(fund) / 12
first_value = fund[f'{fund_name}'].drop_nulls()[0]
last_value = fund[f'{fund_name}'].drop_nulls()[-1]
cagr = (last_value / first_value) ** (1 / years) - 1
print(f'年率リターン: {cagr:.2%}')

vti_vol = np.std(fund['log_diff'].drop_nulls().to_numpy()) * np.sqrt(12)
print(f'ボラティリティ: {vti_vol:.2%}')

sharp_ratio = cagr / vti_vol
print(f'シャープレシオ: {sharp_ratio:.2f}')

年率リターン: 8.67%
ボラティリティ: 11.42%
シャープレシオ: 0.76


In [81]:
df = pl.DataFrame(df)

years = len(df) / 12
first_value = df['vgk_value'][0]
last_value = df['vgk_value'][-1]
cagr = (last_value / first_value) ** (1 / years) - 1
print(f'年率リターン: {cagr:.2%}')

df = df.with_columns([
    pl.col('vgk_value').log().alias('vgk_log_value'),
    pl.col('vpl_value').log().alias('vpl_log_value'),
])
df = df.with_columns([
    pl.col('vgk_log_value').diff().alias('vgk_log_return'),
    pl.col('vpl_log_value').diff().alias('vpl_log_return'),
])
vgk_vol = np.std(df['vgk_log_return'].drop_nulls().to_numpy()) * np.sqrt(12)
print(f'Volatility: {vgk_vol:.2%}')

sharp_ratio = cagr / vgk_vol
print(f'Sharp Ratio: {sharp_ratio:.2f}')

年率リターン: 5.57%
Volatility: 19.34%
Sharp Ratio: 0.29


In [82]:
df = pl.DataFrame(df)

years = len(df) / 12
first_value = df['vpl_value'][0]
last_value = df['vpl_value'][-1]
cagr = (last_value / first_value) ** (1 / years) - 1
print(f'年率リターン: {cagr:.2%}')

df = df.with_columns([
    pl.col('vgk_value').log().alias('vgk_log_value'),
    pl.col('vpl_value').log().alias('vpl_log_value'),
])
df = df.with_columns([
    pl.col('vgk_log_value').diff().alias('vgk_log_return'),
    pl.col('vpl_log_value').diff().alias('vpl_log_return'),
])
vgk_vol = np.std(df['vpl_log_return'].drop_nulls().to_numpy()) * np.sqrt(12)
print(f'Volatility: {vgk_vol:.2%}')

sharp_ratio = cagr / vgk_vol
print(f'Sharp Ratio: {sharp_ratio:.2f}')

年率リターン: 4.99%
Volatility: 16.32%
Sharp Ratio: 0.31


## ポートフォリオのリターン，ボラティリティ

In [86]:
df = df.join(df_month, on='date', how='left').drop_nulls()
df.head()

date,vgk_value,vpl_value,ifree_em_bond_nav,sbi_em_stock_nav,sbi_vti_nav,emaxis_slim_dev_bond_nav
str,f64,f64,f32,f32,f32,f32
"""2021-06""",59.18586,73.049988,12019.0,13201.0,10000.0,11726.0
"""2021-08""",62.05545,73.323189,11896.0,12348.0,10413.0,11801.0
"""2021-09""",58.372158,72.358871,11753.0,12191.0,10219.0,11718.0
"""2021-11""",58.647778,69.209824,11432.0,12216.0,11016.0,11834.0
"""2022-01""",59.490448,67.799835,11726.0,12319.0,10456.0,11698.0


In [87]:
df_log_return = pl.DataFrame({
    'vti_log_return': df['sbi_vti_nav'].log().diff(),
    'vgk_log_return': df['vgk_value'].log().diff(),
    'vpl_log_return': df['vpl_value'].log().diff(),
    'sbi_em_stock_log_return': df['sbi_em_stock_nav'].log().diff(),
}).drop_nulls()

df_log_return.head()

vti_log_return,vgk_log_return,vpl_log_return,sbi_em_stock_log_return
f32,f64,f64,f32
0.04047,0.047346,0.003733,-0.066798
-0.018806,-0.061189,-0.013239,-0.012796
0.0751,0.004711,-0.044495,0.002048
-0.052173,0.014266,-0.020583,0.008396
-0.002394,-0.054337,-0.000269,0.001622


In [ ]:
# 均等加重
weights = np.array([0.30, 0.30, 0.15, 0.25])


tmp = np.array(df_log_return)
cov_matrix = np.cov(tmp, rowvar=False)

returns = np.array()

portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
portfolio_std = np.sqrt(portfolio_variance) * np.sqrt(12)

print(f'ポートフォリオのボラティリティ: {portfolio_std:.2%}')

ポートフォリオのボラティリティ: 12.79%
